In [2]:
# === 0. CÀI ĐẶT THƯ VIỆN ===

!pip install --upgrade pip
!pip install sympy --upgrade
!pip install -q transformers datasets torch pyvi scikit-learn
!pip install pandas transformers torch sentencepiece


  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
Using cached sympy-1.13.3-py3-none-any.whl (6.2 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.13.3 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 182.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 183.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 M

In [3]:
import os
import torch
from transformers import pipeline


In [4]:

# Phát hiện thiết bị
if 'COLAB_TPU_ADDR' in os.environ:
    # TPU
    import torch_xla.core.xla_model as xm
    device = xm.xla_device()
elif torch.cuda.is_available():
    # GPU
    device = torch.device('cuda')
else:
    # CPU
    device = torch.device('cpu')

# Khởi tạo pipeline với device phù hợp
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="wonrax/phobert-base-vietnamese-sentiment",
    tokenizer="wonrax/phobert-base-vietnamese-sentiment",
    device= 0 if str(device).startswith('cuda') else
            -1 if str(device).startswith('cpu') else
            # với TPU, pipeline hiện chưa hỗ trợ trực tiếp XLA, bạn sẽ phải tự viết loop:
            None
)

print("Running on", device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

...

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cuda:0


Running on cuda


In [5]:
# === 1. MOUNT GOOGLE DRIVE (TÙY CHỌN) ===
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:

# Ví dụ đặt data ở: /content/drive/MyDrive/data/all_titles_labeled.csv
DATA_PATH = "/content/drive/MyDrive/data/Vietnamese_newspaper_titles_labeled.csv"
# OUTPUT_DIR = "/content/drive/MyDrive/models/phobert-newspaper-sentiment"
OUTPUT_DIR = "/content/models/phobert-newspaper-sentiment"

In [13]:
OUTPUT_DIR2 = "/content/drive/MyDrive/models/phobert-newspaper-sentiment"

In [7]:

# === 2. IMPORT & SEED ===
import random
import numpy as np
import torch
import transformers

print(f"Transformers version: {transformers.__version__}")

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)


Transformers version: 4.51.1


In [8]:

# === 3. LOAD DỮ LIỆU ===
import pandas as pd
df = pd.read_csv(DATA_PATH)
# Cột tiêu đề: "title", cột nhãn: "sentiment_label" (NEG/POS/NEU)
df['label_id'] = df['sentiment_label'].map({"NEG":0, "POS":1, "NEU":2})

# Chia train/validation
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(
    df[["title","label_id"]],
    test_size=0.15,
    stratify=df["label_id"],
    random_state=SEED
)
print(f"Train: {len(train_df)}, Val: {len(val_df)}")


Train: 3711, Val: 655


In [9]:

# === 4. WORD‑SEGMENTATION VÀ TOKENIZATION ===
from datasets import Dataset
from pyvi import ViTokenizer
from transformers import AutoTokenizer, DataCollatorWithPadding

MODEL_NAME = "wonrax/phobert-base-vietnamese-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

def preprocess_batch(batch):
    # 1) Word‑segmentation
    seg = ViTokenizer.tokenize(batch["title"])
    # 2) Tokenize với PhoBERT tokenizer
    enc = tokenizer(
        seg,
        truncation=True,
        padding=False,
        max_length=64,
    )
    return {
        "input_ids": enc["input_ids"],
        "attention_mask": enc["attention_mask"],
        "labels": batch["label_id"]
    }

# Chuyển sang HuggingFace Dataset
train_ds = Dataset.from_pandas(train_df).map(preprocess_batch, batched=False)
val_ds   = Dataset.from_pandas(val_df).map(preprocess_batch, batched=False)

# Collator để dynamic pad
data_collator = DataCollatorWithPadding(tokenizer)



Map:   0%|          | 0/3711 [00:00<?, ? examples/s]

Map:   0%|          | 0/655 [00:00<?, ? examples/s]

In [10]:
# === 5. SET UP TRAINER ===
from transformers import RobertaForSequenceClassification, TrainingArguments, Trainer, EvalPrediction
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

model = RobertaForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
        "precision": precision_score(labels, preds, average="weighted"),
        "recall": recall_score(labels, preds, average="weighted"),
    }

# Modern transformers 4.x configuration
from transformers import IntervalStrategy

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=12,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    learning_rate=8e-6,
    weight_decay=0.01,
    fp16=True,
    eval_strategy="epoch",            # This is the correct parameter name
    save_strategy="epoch",            # Match save_strategy with eval_strategy
    logging_strategy="steps",         # This is correct
    logging_steps=100,
    save_total_limit=1,               # Keep only the best model
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    seed=SEED,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

<ipython-input-10-0dc903b7c2bb>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:


# === 6. TRAIN ===
trainer.train()



wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: phong02468 (phong02468-fpt-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.546369,0.789313,0.785209,0.787590,0.789313
2,0.647000,0.455998,0.816794,0.814191,0.815958,0.816794
3,0.647000,0.437413,0.829008,0.825786,0.830367,0.829008
4,0.422400,0.417296,0.839695,0.837672,0.839663,0.839695
5,0.422400,0.414289,0.845802,0.844480,0.845038,0.845802
6,0.336200,0.433876,0.839695,0.836828,0.840666,0.839695
7,0.268500,0.443069,0.832061,0.829013,0.833094,0.832061
8,0.268500,0.444577,0.841221,0.838841,0.841488,0.841221
9,0.229800,0.432866,0.847328,0.846498,0.846445,0.847328
10,0.229800,0.436808,0.848855,0.847831,0.847998,0.848855


TrainOutput(global_step=696, training_loss=0.3294032633989707, metrics={'train_runtime': 698.5592, 'train_samples_per_second': 63.748, 'train_steps_per_second': 0.996, 'total_flos': 621820770714486.0, 'train_loss': 0.3294032633989707, 'epoch': 12.0})

In [15]:
# === 7. ĐÁNH GIÁ CUỐI & LƯU MODEL ===
metrics = trainer.evaluate()
print(metrics)

trainer.save_model(OUTPUT_DIR)
print(f"Saved model to {OUTPUT_DIR}")

{'eval_loss': 0.4368078410625458, 'eval_accuracy': 0.8488549618320611, 'eval_f1': 0.8478313777574609, 'eval_precision': 0.8479977971777372, 'eval_recall': 0.8488549618320611, 'eval_runtime': 0.3811, 'eval_samples_per_second': 1718.725, 'eval_steps_per_second': 28.864, 'epoch': 12.0}
Saved model to /content/models/phobert-newspaper-sentiment


In [23]:
from transformers import RobertaForSequenceClassification, AutoConfig

MODEL_NAME = "wonrax/phobert-base-vietnamese-sentiment"

# 1. Load config (chứa thông tin số layers, hidden size, etc.)
config = AutoConfig.from_pretrained(MODEL_NAME)
print("=== Model config ===")
print(f"  • num_hidden_layers: {config.num_hidden_layers}")
print(f"  • hidden_size:       {config.hidden_size}")
print(f"  • num_attention_heads: {config.num_attention_heads}")
print(f"  • intermediate_size: {config.intermediate_size}")

# 2. Load model và đếm số layer thực tế
model = RobertaForSequenceClassification.from_pretrained(MODEL_NAME)
encoder = model.roberta.encoder

print("\n=== Encoder layers ===")
print(f"  • Total layers in encoder: {len(encoder.layer)}")
# In tên class hoặc module của từng layer
for i, layer in enumerate(encoder.layer):
    print(f"    - Layer {i+1}: {layer.__class__.__name__}")


=== Model config ===
  • num_hidden_layers: 12
  • hidden_size:       768
  • num_attention_heads: 12
  • intermediate_size: 3072

=== Encoder layers ===
  • Total layers in encoder: 12
    - Layer 1: RobertaLayer
    - Layer 2: RobertaLayer
    - Layer 3: RobertaLayer
    - Layer 4: RobertaLayer
    - Layer 5: RobertaLayer
    - Layer 6: RobertaLayer
    - Layer 7: RobertaLayer
    - Layer 8: RobertaLayer
    - Layer 9: RobertaLayer
    - Layer 10: RobertaLayer
    - Layer 11: RobertaLayer
    - Layer 12: RobertaLayer


In [24]:
# Freeze 8 first layers
for layer in encoder.layer[:4]:
    for param in layer.parameters():
        param.requires_grad = False

# Chỉ các layer 8–11 được train, cùng với head classification

In [25]:

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
        "precision": precision_score(labels, preds, average="weighted"),
        "recall": recall_score(labels, preds, average="weighted"),
    }

# Modern transformers 4.x configuration
from transformers import IntervalStrategy

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=12,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    learning_rate=8e-6,
    weight_decay=0.01,
    fp16=True,
    eval_strategy="epoch",            # This is the correct parameter name
    save_strategy="epoch",            # Match save_strategy with eval_strategy
    logging_strategy="steps",         # This is correct
    logging_steps=100,
    save_total_limit=1,               # Keep only the best model
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    seed=SEED,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)



# === 6. TRAIN ===
trainer.train()


<ipython-input-25-53cb5ca9ea4b>:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.732500,0.531493,0.789313,0.785837,0.789141,0.789313
2,0.530500,0.457168,0.812214,0.809151,0.812297,0.812214
3,0.460000,0.437580,0.830534,0.827214,0.832592,0.830534
4,0.403700,0.421861,0.836641,0.833687,0.839060,0.836641
5,0.383800,0.423729,0.841221,0.838587,0.844036,0.841221
6,0.336100,0.432891,0.830534,0.826762,0.833047,0.830534
7,0.288000,0.441309,0.836641,0.833653,0.838120,0.836641
8,0.269400,0.435335,0.845802,0.843831,0.845868,0.845802
9,0.249000,0.430042,0.838168,0.836641,0.837605,0.838168
10,0.222300,0.440667,0.839695,0.837784,0.839961,0.839695


TrainOutput(global_step=1392, training_loss=0.3479749800144941, metrics={'train_runtime': 326.4243, 'train_samples_per_second': 136.424, 'train_steps_per_second': 4.264, 'total_flos': 554820891684948.0, 'train_loss': 0.3479749800144941, 'epoch': 12.0})

In [20]:

# === 7. ĐÁNH GIÁ CUỐI & LƯU MODEL ===
metrics = trainer.evaluate()
print(metrics)

trainer.save_model(OUTPUT_DIR2)
print(f"Saved model to {OUTPUT_DIR2}")

{'eval_loss': 0.4185522198677063, 'eval_accuracy': 0.8473282442748091, 'eval_f1': 0.84496131272935, 'eval_precision': 0.8486692540072494, 'eval_recall': 0.8473282442748091, 'eval_runtime': 0.3761, 'eval_samples_per_second': 1741.333, 'eval_steps_per_second': 29.244, 'epoch': 12.0}
Saved model to /content/drive/MyDrive/models/phobert-newspaper-sentiment
